# DEPENDENCIES

In [ ]:
import os
import glob
import xml.etree.ElementTree as ET
import pandas as pd
import tensorflow as tf
print(tf.__version__)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# PATH STRUCTURE

In [ ]:
WORKSPACE_PATH = '/content/drive/MyDrive/Tensorflow_Scratch_Drone'
MODEL_PATH = '/content/drive/MyDrive/Tensorflow_Scratch_Drone/data/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
CONFIG_PATH = MODEL_PATH + 'pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'checkpoint'
ANNOTATION_PATH = WORKSPACE_PATH+'data/Annotations'


In [ ]:
cd /content/drive/MyDrive/Tensorflow_Scratch_Drone

# TRANSFER LEARNING USING TF WEIGHTS

CLONING MODEL ZOO FROM TENSORFLOW OFFICIAL REPO

In [ ]:
!git clone  https://github.com/tensorflow/models

In [ ]:
#wget.download('http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz')
#!mv ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz {PRETRAINED_MODEL_PATH}
#!cd {PRETRAINED_MODEL_PATH} && tar -zxvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

In [ ]:
%cd models/research

In [ ]:
# Define the source tar.gz file and destination directory
source_tar_file = '/content/drive/MyDrive/Tensorflow_Scratch_Drone/data/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz[1].tar'
destination_dir = '/content/drive/MyDrive/Tensorflow_Scratch_Drone/data/'

# Use the tar command to extract the contents
!tar -xvf $source_tar_file -C $destination_dir


In [ ]:
!protoc object_detection/protos/*.proto --python_out=.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

MODEL CHECK

In [ ]:
!python object_detection/builders/model_builder_tf2_test.py

# GENERATE TF RECORD FROM XML(PASCAL VOC) ANNOTATIONS

In [ ]:
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}

# SSD 320x320 FPN LITE

In [ ]:
Mode_config="object_detection/configs/tf2/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config"

In [ ]:
import tensorflow as tf

from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [ ]:
CONFIG_PATH = '/content/drive/MyDrive/Tensorflow_Scratch_Drone/data/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'
print(CONFIG_PATH)
config= config_util.get_configs_from_pipeline_file(CONFIG_PATH)


In [ ]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH,'r') as f:
  proto_str = f.read()
  text_format.Merge(proto_str,pipeline_config)

# CUSTOM PIPELINE CONFIG

In [ ]:
pipeline_config.model.ssd.num_classes = 1
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = 'detection'
pipeline_config.train_input_reader.label_map_path = ANNOTATION_PATH + 'label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:]=[ANNOTATION_PATH+'/Drone.tfrecord']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH+'/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = ANNOTATION_PATH+'/Drone.tfrecord'

In [ ]:
pipeline_config

# UPDATING IN CONFIG FILE

In [ ]:
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(CONFIG_PATH,'wb') as f:
  f.write(config_text)

In [ ]:
#JUST TO SHOW THE CHANGES MADE JUST A MARKDOWN CELL
model {
  ssd {
    num_classes: 1 #CHANGED
    image_resizer {
      fixed_shape_resizer {
        height: 320
        width: 320
      }
    }
    feature_extractor {
      type: "ssd_mobilenet_v2_fpn_keras"
      depth_multiplier: 1.0
      min_depth: 16
      conv_hyperparams {
        regularizer {
          l2_regularizer {
            weight: 4e-05
          }
        }
        initializer {
          random_normal_initializer {
            mean: 0.0
            stddev: 0.01
          }
        }
        activation: RELU_6
        batch_norm {
          decay: 0.997
          scale: true
          epsilon: 0.001
        }
      }
      use_depthwise: true
      override_base_feature_extractor_hyperparams: true
      fpn {
        min_level: 3
        max_level: 7
        additional_layer_depth: 128
      }
    }
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
        use_matmul_gather: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    box_predictor {
      weight_shared_convolutional_box_predictor {
        conv_hyperparams {
          regularizer {
            l2_regularizer {
              weight: 4e-05
            }
          }
          initializer {
            random_normal_initializer {
              mean: 0.0
              stddev: 0.01
            }
          }
          activation: RELU_6
          batch_norm {
            decay: 0.997
            scale: true
            epsilon: 0.001
          }
        }
        depth: 128
        num_layers_before_predictor: 4
        kernel_size: 3
        class_prediction_bias_init: -4.6
        share_prediction_tower: true
        use_depthwise: true
      }
    }
    anchor_generator {
      multiscale_anchor_generator {
        min_level: 3
        max_level: 7
        anchor_scale: 4.0
        aspect_ratios: 1.0
        aspect_ratios: 2.0
        aspect_ratios: 0.5
        scales_per_octave: 2
      }
    }
    post_processing {
      batch_non_max_suppression {
        score_threshold: 1e-08
        iou_threshold: 0.6
        max_detections_per_class: 100
        max_total_detections: 100
        use_static_shapes: false
      }
      score_converter: SIGMOID
    }
    normalize_loss_by_num_matches: true
    loss {
      localization_loss {
        weighted_smooth_l1 {
        }
      }
      classification_loss {
        weighted_sigmoid_focal {
          gamma: 2.0
          alpha: 0.25
        }
      }
      classification_weight: 1.0
      localization_weight: 1.0
    }
    encode_background_as_zeros: true
    normalize_loc_loss_by_codesize: true
    inplace_batchnorm_update: true
    freeze_batchnorm: false
  }
}
train_config {
  batch_size: 8 #CHANGED
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
  data_augmentation_options {
    random_crop_image {
      min_object_covered: 0.0
      min_aspect_ratio: 0.75
      max_aspect_ratio: 3.0
      min_area: 0.75
      max_area: 1.0
      overlap_thresh: 0.0
    }
  }
  sync_replicas: true
  optimizer {
    momentum_optimizer {
      learning_rate {
        cosine_decay_learning_rate {
          learning_rate_base: 0.08
          total_steps: 50000
          warmup_learning_rate: 0.026666
          warmup_steps: 1000
        }
      }
      momentum_optimizer_value: 0.9
    }
    use_moving_average: false
  }
  fine_tune_checkpoint: "/content/drive/MyDrive/Tensorflow_Scratch_Drone/data/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0" #CHANGED
  num_steps: 50000
  startup_delay_steps: 0.0
  replicas_to_aggregate: 8
  max_number_of_boxes: 100
  unpad_groundtruth_tensors: false
  fine_tune_checkpoint_type: "detection" #CHANGED FROM CLASSIFICATION
  fine_tune_checkpoint_version: V2
}
train_input_reader {
  label_map_path: "/content/drive/MyDrive/Tensorflow_Scratch_Drone/data/Annotations/label_map_SSD.pbtxt" #CHANGED
  tf_record_input_reader {
    input_path: "/content/drive/MyDrive/Tensorflow_Scratch_Drone/data/Annotations/train.record" #CHANGED
  }
}
eval_config {
  metrics_set: "coco_detection_metrics"
  use_moving_averages: false
}
eval_input_reader {
  label_map_path: "/content/drive/MyDrive/Tensorflow_Scratch_Drone/data/Annotations/label_map_SSD.pbtxt"
  shuffle: false
  num_epochs: 1
  tf_record_input_reader {
    input_path: "/content/drive/MyDrive/Tensorflow_Scratch_Drone/data/Annotations/train.record"
  }

}


# TRAIN

In [ ]:
%load_ext tensorboard
%tensorboard --logdir '/content/drive/MyDrive/Tensorflow_Scratch_Drone/data/logs_train/'

In [ ]:
!kill 28711

In [ ]:

%cd object_detection

In [ ]:
!python model_main_tf2.py --model_dir=/content/drive/MyDrive/Tensorflow_Scratch_Drone/data/logs_train2 --pipeline_config_path=/content/drive/MyDrive/Tensorflow_Scratch_Drone/data/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8pipeline.config

In [ ]:
!pip install tensorflow==2.13

# LOAD TRAINED MODEL

In [ ]:
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder


In [ ]:

CONFIG_PATH = '/content/drive/MyDrive/Tensorflow_Scratch_Drone/data/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8pipeline.config'
CHECKPOINT_PATH = '/content/drive/MyDrive/Tensorflow_Scratch_Drone/data/logs_train2'

In [ ]:
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'],is_training =False)

ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH,'ckpt-9')).expect_partial()

# TESTING

In [ ]:
@tf.function
def detect_fn(image):
  image,shapes = detection_model.preprocess(image)
  prediction_dict= detection_model.predict(image,shapes)
  detections = detection_model.postprocess(prediction_dict,shapes)
  return detections

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow


In [ ]:
category_index = label_map_util.create_category_index_from_labelmap('/content/drive/MyDrive/Tensorflow_Scratch_Drone/data/Annotations/label_map_SSD.pbtxt')
category_index

In [ ]:
cap = cv2.VideoCapture('/content/drive/MyDrive/Tensorflow_Scratch_Drone/Dronetest.mp4')
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))


In [ ]:
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_file = 'output_video.avi'
output_resolution = (640, 640)

out = cv2.VideoWriter(output_file, fourcc, 20.0, output_resolution)

while True:
    ret, frame = cap.read()
    if not ret:
        break  # End of video stream

    image_np = np.array(frame)
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    num_detections = int(detections.pop('num_detections'))

    detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
    detections['num_detections'] = num_detections
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        detections['detection_boxes'],
        detections['detection_classes'] + label_id_offset,
        detections['detection_scores'],
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=1,
        min_score_thresh=0.1,
        agnostic_mode=False
    )

    # Resize and write the frame to the video output
    out.write(cv2.resize(image_np_with_detections, output_resolution))
    cv2_imshow(cv2.resize(image_np_with_detections, (640, 640)))

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the VideoWriter and destroy all OpenCV windows
out.release()
cv2.destroyAllWindows()